In [ ]:
# for best model, what are the inducing points capturing? these are the I's, rather than the x \dot I's

In [ ]:
# best model is syn_props_scvi (this is trained on B cells vs. Monocytes 0.25 vs 0.1, dir_strength = 50)

In [2]:
import os
import random
from tqdm.notebook import tqdm
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import itertools


%env CUDA_VISIBLE_DEVICES=3
import torch
torch.no_grad()
print("cuda available: " + str(torch.cuda.is_available()))

#import cuml
from torchvision.utils import save_image, make_grid


env: CUDA_VISIBLE_DEVICES=3
cuda available: True


In [3]:
from matplotlib import rcParams, rcParamsDefault
rcParams.update(rcParamsDefault)
plt.rcParams.update({'font.size': 15})

In [7]:
save_dir = '../figures/images'
experiment_name = 'syn_props_hdim32_scvi'
summary_name = os.path.join('/home/rpeyser/GitHub/scSet/checkpoints/', experiment_name, 'summary.pth')

imgdir = os.path.join(save_dir, experiment_name)
imgdir_gt = os.path.join(imgdir, 'gt')
imgdir_recon = os.path.join(imgdir, 'recon')
imgdir_gen = os.path.join(imgdir, 'gen')

os.makedirs(save_dir, exist_ok=True)
os.makedirs(imgdir_gt, exist_ok=True)
os.makedirs(imgdir_recon, exist_ok=True)
os.makedirs(imgdir_gen, exist_ok=True)

In [8]:
summary = torch.load(summary_name)
for k, v in summary.items():
    try:
        print(f"{k}: {v.shape}")
    except AttributeError:
        print(f"{k}: {len(v)}")

smp_set: torch.Size([100, 700, 20])
smp_mask: torch.Size([100, 700])
smp_att: 5
priors: 6
recon_set: torch.Size([100, 700, 20])
recon_mask: torch.Size([100, 700])
posteriors: 6
dec_att: 5
enc_att: 5
enc_hiddens: 3
init_set: torch.Size([100, 700, 32])
dec_hiddens: 3
dec_latents: 3
gt_set: torch.Size([100, 503, 20])
gt_mask: torch.Size([100, 503])
mean: torch.Size([3])
std: torch.Size([3])
sid: 3
mid: 3
pid: 3
cardinality: 3


In [11]:
# load the model and access the inducing pts weights (since the inducing points are not a per input item)
model = torch.load("/home/rpeyser/GitHub/scSet/checkpoints/syn_props_hdim32_scvi/checkpoint-best.pt")

In [13]:
for name, param in model.named_parameters():
    print(name)

AttributeError: 'dict' object has no attribute 'named_parameters'

In [15]:
model.keys()

dict_keys(['epoch', 'model', 'optimizer', 'scheduler'])

In [ ]:
#load a trained model from a checkpoint

#the model needs args passed in as a dict so figure out how to parse them from the script, or maybe just build the dict manually here

from args import get_args
args = get_args()
model = SetVAE(args)
model = model.cuda()
save_dir = Path(args.log_dir) / "checkpoints" / args.model_name
# default to checkpoint-best
if args.resume_checkpoint is None and Path(Path(save_dir) / 'checkpoint-best.pt').exists():
    args.resume_checkpoint = os.path.join(save_dir, f'checkpoint-best.pt')
checkpoint = torch.load(args.resume_checkpoint)
model.load_state_dict(checkpoint['model'])
model.eval()